In [1]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os
import warnings
import time
import matplotlib.pyplot as plt
from importlib import reload

reload(Model)
warnings.simplefilter('ignore', FutureWarning)

# positive numbers, start should be lower than end
# a rebalance index represents the month before the rebalancing takes place
# so returns are calculated starting at rebalanceIndex + 1
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    return (indexes)


# get ln returns for an equally balanced portfolio of stocks
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	if len(portfolio) - nancount == 0:
		total = 0
	else:
		total = returns/(len(portfolio)-nancount)
	print("Start Date: " + str(Model.convertIndexToDate(-1 * index + 1)))
	print("Total Return: " +str(total))
	return (total)


# make a portfolio with predicted probabilities higher than a hardcoded threshold
def makePortfolio(treeTuple):
	featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Health Care")
	probabilityThreshold = 0.8
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	if len(stockTuples) == 0:
		print("No portfolio, probabilities lower than threshold of " + str(probabilityThreshold))
		return 0
	stocks, probabilities = zip(*stockTuples)
	return(getReturns(stocks, treeTuple[0], 3))

In [2]:
indexes = []
for i in rebalanceIndexes(4,56):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

18
[array([-219, -216, -213, -210, -207, -204, -201, -198, -195, -192, -189,
       -186, -183, -180, -177, -174, -171, -168, -165, -162, -159, -156,
       -153, -150, -147, -144, -141, -138, -135, -132, -129, -126, -123,
       -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,  -90,
        -87,  -84,  -81,  -78,  -75,  -72,  -69,  -66,  -63,  -60,  -57,
        -54,  -51,  -48,  -45,  -42,  -39,  -36,  -33,  -30,  -27,  -24,
        -21]), array([-222, -219, -216, -213, -210, -207, -204, -201, -198, -195, -192,
       -189, -186, -183, -180, -177, -174, -171, -168, -165, -162, -159,
       -156, -153, -150, -147, -144, -141, -138, -135, -132, -129, -126,
       -123, -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,
        -90,  -87,  -84,  -81,  -78,  -75,  -72,  -69,  -66,  -63,  -60,
        -57,  -54,  -51,  -48,  -45,  -42,  -39,  -36,  -33,  -30,  -27,
        -24]), array([-225, -222, -219, -216, -213, -210, -207, -204, -201, -198, -195,
       -192, -189

In [3]:
print("# of multiprocess cpus: " + str(os.cpu_count()))
sector = "Health Care"
featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
forestList = []
for ind in indexes:
	randForest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = ind, target= 'Rate of Return', features = featureList, featureLength = 12,\
									targetLength = 3, sector = sector, percentileTarget = 90, percentileAvoid = 10, verbose = True)
	forestList.append(randForest)
print(forestList)

# of multiprocess cpus: 8
Finished data retrieval, starting model training. Time taken: 114.11086511611938 seconds.
Finished fitting. Time taken: 21.451729774475098 seconds.
Finished data retrieval, starting model training. Time taken: 121.69834685325623 seconds.
Finished fitting. Time taken: 20.179762840270996 seconds.
Finished data retrieval, starting model training. Time taken: 195.98341393470764 seconds.
Finished fitting. Time taken: 20.897146224975586 seconds.
Finished data retrieval, starting model training. Time taken: 123.25280117988586 seconds.
Finished fitting. Time taken: 20.49976897239685 seconds.
Finished data retrieval, starting model training. Time taken: 122.03915691375732 seconds.
Finished fitting. Time taken: 20.657044887542725 seconds.
Finished data retrieval, starting model training. Time taken: 125.05518293380737 seconds.
Finished fitting. Time taken: 20.61947798728943 seconds.
Finished data retrieval, starting model training. Time taken: 114.73440599441528 seconds

In [4]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,56), forestList))
print(returnsList)

# of multiprocess cpus: 8
('APPF', 'ETSY', 'TTGT')
Total Return: 0.07537432604755108
-3
-1
APPF: -0.018713996431373392
ETSY: 0.1307937483291548
TTGT: 0.11404322624487184
Start Date: 2017-11-30
('ANET', 'APPF', 'CRCM', 'FSLR', 'IPGP', 'LLNW', 'MTCH', 'NAME', 'QLYS', 'QNST', 'SHOR', 'SQ', 'TEAM', 'TYPE')
FSLR: 0.15475891535479835
-6
-4
ANET: 0.12643131045530875
APPF: 0.06759329113252832
CRCM: 0.026369359118967317
IPGP: 0.19157944039320096
LLNW: 0.3212353740804115
MTCH: 0.20654681413910403
NAME: 0.17953659378446796
QLYS: 0.10767362782365808
QNST: 0.508954715830179
SHOR: -0.12160713209478691
SQ: 0.35372152522594114
TEAM: 0.3065341489769069
TYPE: 0.23350068941415714
Start Date: 2017-08-31
Total Return: 0.1902020481167745
('AAOI', 'ALRM', 'APPF', 'BRKS', 'CRCM', 'CTRL', 'FN', 'FSLR', 'GAME', 'N', 'NSR')
Total Return: 0.10167929799725385
-9
-7
AAOI: 0.33382930696429014
ALRM: 0.15559795328559023
APPF: 0.19876502551907071
BRKS: -0.11488343231998188
CRCM: -0.056202079968312724
CTRL: 0.1407471102